In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def read_xlsx_and_plot(csv_path, label):
    # 读取 & 列名统一小写
    df_1 = pd.read_excel(csv_path, sheet_name="H")
    df_1.columns = df_1.columns.str.lower()
    if "avg" not in df_1.columns:
        # 容错：找一个与 avg 同名（忽略大小写）的列
        col_avg = next(c for c in df_1.columns if c.lower() == "avg")
    else:
        col_avg = "avg"
    df_1 = df_1[[col_avg]].rename(columns={col_avg: "avg"})

    df_2 = pd.read_excel(csv_path, sheet_name="Hori")
    df_2.columns = df_2.columns.str.lower()
    if "hori" not in df_2.columns:
        col_hori = next(c for c in df_2.columns if c.lower() == "hori")
    else:
        col_hori = "hori"
    df_2 = df_2[[col_hori]].rename(columns={col_hori: "hori"})

    # 对齐长度
    min_len = min(len(df_1), len(df_2))
    df_1 = df_1.iloc[:min_len]
    df_2 = df_2.iloc[:min_len]
    dfs = pd.concat([df_1, df_2], axis=1)

    x = dfs["avg"].to_numpy()
    y = dfs["hori"].to_numpy()

    # ——（关键1）自动搜索最优 k：前段二次，后段二次（或一次）——
    def piecewise_yhat(x, y, k, deg_back=2, w=None):
        # 权重按段切
        w1 = None if w is None else w[:k]
        w2 = None if w is None else w[k:]
        c1 = np.polyfit(x[:k], y[:k], 2, w=w1)
        c2 = np.polyfit(x[k:], y[k:], deg_back, w=w2)
        y1 = np.polyval(c1, x[:k])
        y2 = np.polyval(c2, x[k:])
        return np.concatenate([y1, y2]), (c1, c2)

    # ——（关键2）给高湿区稍加权，提高后半段贴合度——
    # 简单线性权重：湿度越高，权重越大（1 ~ 3 可调）
    if x.max() > x.min():
        w = 1.0 + 2.0 * (x - x.min()) / (x.max() - x.min())  # [1,3]
    else:
        w = np.ones_like(x)

    # 搜索 k（你之前 1000 左右，这里在 [800, 1400] 范围搜；步长可调）
    candidates = range(800, min(len(x)-3, 1400), 20)  # 确保两段各≥3点
    best = None
    for k in candidates:
        yhat, _ = piecewise_yhat(x, y, k, deg_back=2, w=w)
        rmse = float(np.sqrt(np.mean((y - yhat) ** 2)))
        if best is None or rmse < best[0]:
            best = (rmse, k, yhat)
    rmse_best, k_star, yhat_best = best
    dfs["new"] = yhat_best

    # 残差（可选）：如果你还想再压一压，可以分段一次/二次修正；
    # 但很多情况下上面这步已经比固定k+普通polyfit好不少了。
    res = y - dfs["new"].to_numpy()

    # ——（可选）残差分段轻度修正（仍用 polyfit；不想要可以注释掉）——
    # 前段二次，后段一次；并在 k 附近做个小过渡避免折痕
    k = k_star
    bw = 50  # 过渡带
    c_res1 = np.polyfit(x[:k],  res[:k],  2)
    c_res2 = np.polyfit(x[k:],  res[k:],  1)
    r1 = np.polyval(c_res1, x[:k])
    r2 = np.polyval(c_res2, x[k:])
    trend = np.empty_like(res)
    # 平滑拼接
    trend[:k-bw] = r1[:k-bw]
    trend[k+bw:] = r2[bw:]
    alpha = np.linspace(0, 1, 2*bw, endpoint=True)
    trend[k-bw:k+bw] = (1-alpha) * r1[k-bw:k+bw] + alpha * r2[:2*bw]

    dfs["twice"] = dfs["new"] + trend
    res_after = y - dfs["twice"].to_numpy()

    # 画图
    plt.figure(figsize=(12,5))
    plt.plot(dfs["avg"],  label="avg_humidity")
    plt.plot(dfs["hori"], label="ori_humidity", alpha=0.6)
    plt.plot(dfs["twice"], label="piecewise_w_fitted")
    plt.legend(); plt.grid(True); plt.xlabel("Times"); plt.ylabel("%RH")
    plt.title(f"Humidity(ori vs. self {label}) - k*={k_star}, RMSE={rmse_best:.4f}")
    plt.tight_layout()
    plt.savefig(f"初始对比图 {label}.png")
    plt.show()

    # 差分图
    diff = dfs["hori"] - dfs["avg"]
    plt.figure(figsize=(12,5))
    plt.plot(diff, alpha=0.6, label="ori - avg")
    plt.plot(res_after, label="residual after correction")
    plt.legend(); plt.grid(True); plt.xlabel("Times"); plt.ylabel("Δ")
    plt.title(f"Humidity Difference {label}")
    plt.tight_layout()
    plt.savefig(f"Difference {label}.png")
    plt.show()

    return dfs

print(read_xlsx_and_plot("40C_xlsx/40C40H.xlsx", label = "40C40H"))
print(read_xlsx_and_plot("40C_xlsx/40C60H.xlsx", label = "40C60H"))
print(read_xlsx_and_plot("40C_xlsx/40C80H.xlsx", label = "40C80H"))
print(read_xlsx_and_plot("40C_xlsx/40C95H.xlsx", label = "40C95H"))


ValueError: operands could not be broadcast together with shapes (100,) (50,) 